# Ejercicio 12: Web Scraping
## Objetivo de la práctica
El objetivo de este ejercicio es construir un web scraper que recoja datos de un website.
### Parte 0: Planificar
1. Identificar los datos que quieres obtener.
2. Elegir el sitio web objetivo.
3. Planificar la estructura del corpus.

### Parte 1: Entender el sitio web objetivo
- Analizar la estructura de la página web a ser analizada.
- Identificar los elementos HTML que contienen los datos buscados.

In [ ]:
# Obtener el archivo HTML
!wget -O rotisserie-chicken.html \
--header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36" \
"https://www.allrecipes.com/recipe/93168/rotisserie-chicken/"

In [ ]:
from bs4 import BeautifulSoup

file = 'rotisserie-chicken.html'

# load html file
with open(file, "r", encoding="UTF-8") as file:
    html_content = file.read()

# parse the html content with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

In [ ]:
# extracting the recipe title
title = soup.find("meta", {"property":"og:title"})["content"]
title

In [ ]:
# Ingredients
ingredients_section = soup.find_all("li", class_ = "mm-recipes-structured-ingredients__list-item")
for ingredient in ingredients_section:
    print(ingredient.text.strip())

### Parte 2: Obtener los datos deseados
- Buscar dentro del contenido HTML y extraer la información.

In [ ]:
# Extracting the summary
summary= soup.find("p", class_ = "article-subheading text-utility-300").text.strip()

In [ ]:
# Extracting the ingredients
ingredients_section = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
ingredients = [ingredient.get_text().strip() for ingredient in ingredients_section]

In [ ]:
# Extracting the rating
review= soup.find("div", class_ = "comp mm-recipes-review-bar__rating mntl-text-block text-label-300").text.strip()

In [ ]:
# Extracting the number of Servings
import re

serving_results = soup.find_all("div", class_="mm-recipes-details__value")

for serving in serving_results:
    text = serving.text.strip()
    if re.fullmatch(r"\d+", text):  # Solo si es un número entero
        servings = text

In [ ]:
# Extracting the time

# todos los items de detalle
details = soup.find_all("div", class_="mm-recipes-details__item")

for item in details:
    label = item.find("div", class_="mm-recipes-details__label")
    value = item.find("div", class_="mm-recipes-details__value")
    # extract the time
    if label and label.text.strip() == "Total Time:":
        time = value.text.strip()

In [ ]:
# directions section
li_items = soup.find_all("li", class_="comp mntl-sc-block mntl-sc-block-startgroup mntl-sc-block-group--LI")
directions = []
# Itera sobre ellos y busca su <p> hijo
for li in li_items:
    p_tag = li.find("p")
    if p_tag:
        directions.append(p_tag.text.strip())

In [ ]:
# Extracting the nutrition information
nutrition_section = soup.find_all("span", class_="mm-recipes-nutrition-facts-label__nutrient-name mm-recipes-nutrition-facts-label__nutrient-name--has-postfix")
nutrition_facts = [fact.parent.get_text().strip().replace('\n', ' ') for fact in nutrition_section]

In [ ]:
# Extracting the image
def extraer_url_imagen_receta(soup):

    article_content = soup.find('div', class_='loc article-content')

    if not article_content:
        return None

    image_url = None

    # Buscar la URL en la etiqueta de video
    video_tag = article_content.find('video')
    if video_tag:
        if video_tag.has_attr('data-poster'):
            image_url = video_tag['data-poster']
        elif video_tag.has_attr('poster'):
            image_url = video_tag['poster']

    # Si no se encontró en el video, buscar en la etiqueta de imagen
    if not image_url:
        figure_tag = article_content.find('figure')
        if figure_tag:
            try:
                img_tag = figure_tag.find('div').find('div').find('img')
                if img_tag and img_tag.has_attr('src'):
                    image_url = img_tag['src']
            except AttributeError:
                pass

    return image_url

In [ ]:
url_imagen = extraer_url_imagen_receta(soup)

print(f"URL extraída: {url_imagen}")

In [ ]:
# Print the extracted information
print("Title:", title)
print("Summary:", summary)
print("Ingredients:")
for ingredient in ingredients:
    print("-", ingredient)
print("Rating:", review)
print("Servings:", servings)
print("Time:", time)
print("Directions:")
for i, direction in enumerate(directions, 1):
    print(f"{i}." + direction)
print("Nutrition Facts:")
for fact in nutrition_facts:
    print("-", fact)
print("Image URL:", url_imagen)

### Parte 3: Obtener enlaces relacionados
- Encontrar links a otras recetas para completar el corpus

In [ ]:
# Lista para almacenar las URLs encontradas
recipe_links = []

# Encontrar todos los hipervínculos (<a>) con la clase específica de las tarjetas de recetas
link_elements = soup.find_all('a', class_='comp mntl-card-list-items mntl-universal-card mntl-document-card mntl-card card card--no-image')

# Iterar sobre los elementos encontrados y extraer la URL
for link in link_elements:
    # Obtener el valor del atributo 'href', que contiene la URL
    href = link.get('href')

    # Validar que sea un enlace de receta y no esté vacío
    # Nos aseguramos que el enlace existe y que es una receta (suelen empezar con '/recipe/').
    if href and href.startswith('https://www.allrecipes.com/recipe/'):
        recipe_links.append(href)

In [ ]:
# Mostrar los resultados
print(f"✅ Se encontraron {len(recipe_links)} enlaces a recetas.")
for url in recipe_links:
    print(url)

In [ ]:
import requests
import time

def crear_corpus_recetas(url_inicial, cantidad_maxima=100):

    # La cola de URLs que necesitamos visitar. Empezamos con la URL inicial.
    urls_a_visitar = [url_inicial]

    # Un conjunto (set) para guardar las URLs que ya hemos visitado o agregado a la cola.
    urls_visitadas = {url_inicial}

    # La lista final donde guardaremos las recetas válidas encontradas.
    enlaces_recetas_encontrados = []

    print(f"🤖 Iniciando crawler en: {url_inicial}")
    print("-------------------------------------------------")

    # BUCLE PRINCIPAL DEL CRAWLER
    # El bucle se ejecuta mientras tengamos URLs en la cola y no hayamos alcanzado nuestro objetivo.
    while urls_a_visitar and len(enlaces_recetas_encontrados) < cantidad_maxima:

        # Sacamos la primera URL de la lista para procesarla.
        url_actual = urls_a_visitar.pop(0)

        # Añadimos la URL actual a nuestra lista final de recetas.
        enlaces_recetas_encontrados.append(url_actual)
        print(f"[{len(enlaces_recetas_encontrados)}/{cantidad_maxima}] Procesando: {url_actual}")

        try:
            # OBTENER Y PARSEAR EL HTML

            headers = {'User-Agent': 'RecipeCorpusCrawler/1.0'}

            response = requests.get(url_actual, headers=headers, timeout=10)

            # Si la solicitud no fue exitosa, saltamos a la siguiente URL.
            if response.status_code != 200:
                print(f"  -> Error: No se pudo acceder a la URL (Código: {response.status_code})")
                continue

            soup = BeautifulSoup(response.content, 'html.parser')

            # EXTRAER NUEVOS ENLACES DE LA PÁGINA ACTUAL
            nuevos_enlaces = soup.find_all('a', class_='comp mntl-card-list-items mntl-universal-card mntl-document-card mntl-card card card--no-image')
            print(f"  -> Se encontraron {len(nuevos_enlaces)} nuevos enlaces.")

            for link in nuevos_enlaces:
                href = link.get('href')

                # Verificamos que el enlace es válido, es una receta, y no lo hemos visitado antes.
                if href and href.startswith('https://www.allrecipes.com/recipe/') and href not in urls_visitadas:
                    # Si es un enlace nuevo y válido, lo añadimos a la cola y al conjunto de visitados.
                    urls_visitadas.add(href)
                    urls_a_visitar.append(href)

        except requests.exceptions.RequestException as e:
            print(f"  -> Error de red al intentar acceder a {url_actual}: {e}")

        # Hacemos una pausa de 1 segundo entre cada solicitud para no sobrecargar el sitio.
        time.sleep(1)

    print("-------------------------------------------------")
    print(f"✅ Proceso completado. Total de enlaces recolectados: {len(enlaces_recetas_encontrados)}")
    return enlaces_recetas_encontrados

In [ ]:
# --- EJECUCIÓN DEL CRAWLER
url_semilla = 'https://www.allrecipes.com/recipe/93168/rotisserie-chicken/'
corpus_final = crear_corpus_recetas(url_semilla, cantidad_maxima=100)

# primeros 10 enlaces del corpus final
print("\nPrimeros 10 enlaces del corpus:")
for url in corpus_final[:10]:
    print(url)

In [ ]:
# from each url in corpus_final, extract the tittle, summary, ingredients, rating, servings, time, directions, nutrition facts and image URL to agroup them in a dataframe
import pandas as pd

def scrape_recipe_details(url):
    """
    Visita la URL de una receta y extrae todos sus detalles.
    Retorna un diccionario con la información de la receta.
    """
    try:
        headers = {'User-Agent': 'RecipeScraper/2.0'}
        response = requests.get(url, headers=headers, timeout=15)
        if response.status_code != 200:
            print(f"  -> Error al acceder a {url}, Código: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"  -> Error de red para {url}: {e}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    recipe_data = {}

    try:
        # Title
        recipe_data['title'] = soup.find("meta", {"property":"og:title"})["content"]
    except AttributeError:
        recipe_data['title'] = None

    try:
        # Summary
        recipe_data['summary'] = soup.find("p", class_="article-subheading text-utility-300").text.strip()
    except AttributeError:
        recipe_data['summary'] = None

    try:
        # Ingredients
        ingredients_section = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
        recipe_data['ingredients'] = [ing.get_text(strip=True) for ing in ingredients_section]
    except:
        recipe_data['ingredients'] = []

    try:
        # Rating
        recipe_data['rating'] = soup.find("div", class_="comp mm-recipes-review-bar__rating mntl-text-block text-label-300").text.strip()
    except AttributeError:
        recipe_data['rating'] = None

    try:
        # Servings y Time
        servings, total_time = None, None
        details_items = soup.find_all("div", class_="mm-recipes-details__item")
        for item in details_items:
            label = item.find("div", class_="mm-recipes-details__label").text.strip()
            value = item.find("div", class_="mm-recipes-details__value").text.strip()
            if label == "Servings:":
                servings = value
            elif label == "Total Time:":
                total_time = value
        recipe_data['servings'] = servings
        recipe_data['time'] = total_time
    except:
        recipe_data['servings'] = None
        recipe_data['time'] = None

    try:
        # Directions
        li_items = soup.find_all("li", class_="comp mntl-sc-block mntl-sc-block-startgroup mntl-sc-block-group--LI")
        directions = [li.find("p").text.strip() for li in li_items if li.find("p")]
        recipe_data['directions'] = directions
    except:
        recipe_data['directions'] = []

    try:
        # Nutrition Facts
        nutrition_section = soup.find_all("tr", class_="mm-recipes-nutrition-facts-summary__table-row")
        nutrition_facts = [fact.get_text(strip=True).replace('\n', ' ').replace('  ', ' ') for fact in nutrition_section]
        recipe_data['nutrition_facts'] = nutrition_facts
    except:
        recipe_data['nutrition_facts'] = []

    # Image URL
    recipe_data['image_url'] = extraer_url_imagen_receta(soup)

    # URL de la receta
    recipe_data['source_url'] = url

    return recipe_data

In [ ]:
all_recipes_data = []

print(f"🍲 Empezando a scrapear {len(corpus_final)} recetas...")

# Iteramos sobre cada URL en nuestro corpus
for i, url in enumerate(corpus_final):
    print(f"Procesando [{i+1}/{len(corpus_final)}]: {url}")

    data = scrape_recipe_details(url)

    # Si la función devolvió datos, los añadimos a nuestra lista
    if data:
        all_recipes_data.append(data)

    # para no saturar el servidor
    time.sleep(1)

print("\n✅ Scraping completado.")

# Crear el DataFrame a partir de la lista de diccionarios
df_recetas = pd.DataFrame(all_recipes_data)

print(" DataFrame creado exitosamente:")
print("\nInformación del DataFrame:")
df_recetas.info()

print("\nPrimeras 5 filas del DataFrame:")
print(df_recetas.head())

In [ ]:
df_recetas

#### Preprocesamiento del texto previo al cálculo de embeddings

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [ ]:
def preprocesar_texto(texto):
    """
    Realiza preprocesamiento del texto:
    - Unir palabras cortadas por guión y salto de línea.
    - Eliminar saltos de línea y tabuladores restantes.
    - Eliminar caracteres especiales.
    - Convertir a minúsculas.
    - Tokenizar.
    - Eliminar stopwords.
    - Aplicar stemming.
    Retorna un string con las palabras procesadas separadas por espacio.
    """
    if not texto:
        return ""

    # Unir palabras separadas por guión y salto de línea
    texto = re.sub(r"-\n([a-z])", r"\1", texto)

    # Eliminar saltos de línea y tabuladores sobrantes
    texto = texto.replace("\n", " ").replace("\t", " ")

    # Eliminar caracteres especiales (conservar solo letras y números)
    texto = re.sub(r"[^a-zA-Z0-9 ]", " ", texto)

    # Pasar a minúsculas
    texto = texto.lower()

    # Tokenizar
    tokens = word_tokenize(texto)

    # Eliminar stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Unir tokens de nuevo
    texto_procesado = " ".join(tokens)

    return texto_procesado

In [ ]:
df_recetas["contenido_preprocesado"] = df_recetas.apply(
    lambda row: preprocesar_texto(
        f"{str(row['title'] or '')}. "
        f"{str(row['summary'] or '')}. "
        f"Ingredientes: {' '.join(row['ingredients'] or [])}. "
        f"Instrucciones: {' '.join(row['directions'] or [])}. "
        f"NutritionFacts: {' '.join(row['nutrition_facts'] or [])}"
    ),
    axis=1
)

In [ ]:
df_recetas

#### Obtener embeddings para cada receta
##### Carga del modelo

In [ ]:
from sentence_transformers import SentenceTransformer

# Cargar el modelo
model = SentenceTransformer('all-MiniLM-L6-v2')

##### Cálculo de embeddings

In [ ]:
def generar_embeddings(df, columna_texto="contenido_preprocesado"):
    """
    Genera embeddings SBERT para cada fila del DataFrame.
    Parámetros:
        df: DataFrame con la columna de texto preprocesado.
        columna_texto: nombre de la columna con el texto (por defecto 'contenido_preprocesado').
    Retorna:
        DataFrame con nueva columna 'embedding'.
    """
    # Lista de textos
    textos = df[columna_texto].tolist()

    # Generar embeddings
    embeddings = model.encode(textos, show_progress_bar=True, convert_to_numpy=True)

    # Asignar embeddings al DataFrame
    df["embedding"] = embeddings.tolist()

    return df

In [ ]:
df_recetas = generar_embeddings(df_recetas)

In [ ]:
df_recetas

### Parte 4: Hacer RAG con las recetas obtenidas
- Una vez que se ha construido el corpus, implementar y desplegar RAG para realizar búsquedas en el corpus

##### Similitud coseno y ranking top n

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def obtener_indices_top_similares(df, query_embedding, top_n=5):
    """
    Calcula similitud coseno y devuelve los índices de las filas más similares.
    """

    matriz_embeddings = np.vstack(df["embedding"].values)

    query_embedding = np.array(query_embedding).reshape(1, -1)

    # Calcular similitud coseno
    similitudes = cosine_similarity(query_embedding, matriz_embeddings)[0]

    # Obtener índices ordenados de mayor a menor similitud
    indices_ordenados = np.argsort(similitudes)[::-1]

    # Seleccionar top N
    indices_top_n = indices_ordenados[:top_n]

    return indices_top_n

#### Selección de la receta que desea

In [ ]:
!pip install ipywidgets

In [ ]:
import ipywidgets as widgets
from IPython.display import display

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
def construir_contexto_receta(df, indice):
    receta = df.iloc[indice]

    # Manejo de valores None
    title = receta['title'] or "No disponible"
    summary = receta['summary'] or "No disponible"
    rating = receta['rating'] or "Sin calificación"
    servings = receta['servings'] or "No especificado"
    time = receta['time'] or "No especificado"
    image_url = receta['image_url'] or None

    # Formateo de listas
    ingredients = "\n".join([f"{i+1}. {ing}" for i, ing in enumerate(receta['ingredients'])]) if receta['ingredients'] else "No especificado"
    directions = "\n".join([f"**Paso {i+1}:** {paso}" for i, paso in enumerate(receta['directions'])]) if receta['directions'] else "No disponible"
    nutrition_facts = "\n".join([f"- {fact}" for fact in receta['nutrition_facts']]) if receta['nutrition_facts'] else "No disponible"

    # Construcción del contexto en formato Markdown
    contexto = f"""
## 🍳 {title}

{f'<img src="{image_url}" width="300">' if image_url else '*Sin imagen disponible*'}

**📝 Resumen:** {summary}

**⭐ Calificación:** {rating} estrellas
**👥 Porciones:** {servings}
**⏱️ Tiempo:** {time}

**🥕 Ingredientes:**\n
    {ingredients}

**👨‍🍳 Preparación:**\n
    {directions}

**📊 Datos nutricionales:**\n
    {nutrition_facts}
    """

    return contexto

In [ ]:
query = "barbecue chicken"
preprocessed_query = preprocesar_texto(query)
query_embedding = model.encode(preprocessed_query)

indices_top5 = obtener_indices_top_similares(df_recetas, query_embedding, top_n=5)
print("Índices top 5:", indices_top5)

In [ ]:
indice_seleccionado = None

# Crear opciones para los radio buttons (títulos de las recetas)
opciones_recetas = [f"{idx + 1}: {df_recetas.iloc[idx]['title']}" for idx in indices_top5]

# Crear los widgets (radio buttons + botón)
radio = widgets.RadioButtons(
    options=opciones_recetas,
    description='Elige una receta:',
    disabled=False
)

boton = widgets.Button(description="Seleccionar receta")

def on_button_click(b):
    global indice_seleccionado

    receta_seleccionada = radio.value
    indice_seleccionado = int(receta_seleccionada.split(":")[0]) - 1

    # Bloquear los widgets después de la selección
    radio.disabled = True
    boton.disabled = True

    print("\n--- Receta seleccionada ---")
    print(f"Título: {df_recetas.iloc[indice_seleccionado]['title']}")

boton.on_click(on_button_click)

# Mostrar el formulario
display(radio)
display(boton)

In [ ]:
# Después de que el usuario seleccione una receta (indice_seleccionado contiene el índice)
contexto_receta = construir_contexto_receta(df_recetas, indice_seleccionado)

#### Usar la API para generar la respuesta
Usar las API de deepseek  para la generación de respuestas en base al contexto proporcionado.

In [ ]:
from openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
client_deepseek = OpenAI(api_key="api_key", base_url="URL")

In [ ]:
prompt = f"""
    Eres un asistente culinario experto. Responde usando SOLAMENTE el siguiente contexto.
    Si la pregunta no puede responderse con esta información, di: 'No tengo información sobre tu búsqueda en la receta seleccionada'.

    Instrucciones de formato:
    1. Muestra toda la información de la receta en formato Markdown si es que está disponible dentro del contexto, indícale al usuario que esos son los detalles de la receta y que disfrute de su platillo.
    2. Si no está disponible, responde que no dispones de la información en la receta seleccionada

    ------
    {contexto_receta}
    ------

    Pregunta del usuario: {query}
    """

# Enviar al modelo
response = client_deepseek.chat.completions.create(
    model="deepseek/deepseek-r1:free",
    messages=[
        {"role": "system", "content": "Eres un chef experto que responde con precisión."},
        {"role": "user", "content": prompt}
    ],
    temperature=0
)

# Mostrar resultados
display(Markdown(response.choices[0].message.content))